In [1]:
import cPickle
from PIL import Image
import os
import sys
import numpy as np
from matplotlib.pyplot import imshow
import tensorflow as tf
import random


data_folder = 'cifar-10-batches-py'
train_batch_1 = os.path.join(data_folder, 'data_batch_1')
train_batch_2 = os.path.join(data_folder, 'data_batch_2')
train_batch_3 = os.path.join(data_folder, 'data_batch_3')
train_batch_4 = os.path.join(data_folder, 'data_batch_4')
train_batch_5 = os.path.join(data_folder, 'data_batch_5')
test_batch = os.path.join(data_folder, 'test_batch')

def read_batch(filename):
    with open(filename, 'r') as f:
        data = cPickle.load(f)
    return data

train_filenames = [train_batch_1, train_batch_2, train_batch_3, train_batch_4, train_batch_5]
train_data_arr = []
train_labels_arr = []
for filename in train_filenames:
    d = read_batch(filename)
    data = d['data']
    labels = d['labels']
    train_data_arr.append(data)
    train_labels_arr.append(np.asarray(labels, dtype=np.int32))
train_data = np.vstack(train_data_arr)
train_labels = np.hstack(train_labels_arr)
train_data = np.asarray(train_data, dtype=np.float32)
train_labels = np.asarray(train_labels, dtype=np.int64)
print train_data.shape
print train_labels.shape

test_d = read_batch(test_batch)
test_data = test_d['data']
test_data = np.asarray(test_data, dtype=np.float32)
test_labels = np.asarray(test_d['labels'], dtype=np.int64)
print test_data.shape
print test_labels.shape

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [7]:
batch_size = 20
dataset = tf.data.Dataset.from_tensor_slices((train_data[0:200], train_labels[0:200]))
dataset = dataset.batch(batch_size)
dataset = dataset.repeat()
# iterator = dataset.make_one_shot_iterator()
iterator = dataset.make_initializable_iterator()
batch_imgs, batch_labels = iterator.get_next()


In [3]:
def cnn_model(x, y):
    with tf.name_scope("inputs"):
        is_training = tf.placeholder(tf.bool, name="is_training")
        x_image = tf.reshape(x, [-1, 3, 32, 32])
        x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])
    with tf.name_scope("model"):
        conv1 = tf.layers.conv2d(inputs=x_image,
                                 filters=32,
                                 kernel_size=[5,5],
                                 padding='same',
                                 activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size=[2,2],
                                        strides=2)
        
        conv2 = tf.layers.conv2d(inputs=pool1,
                                 filters=64,
                                 kernel_size=[5,5],
                                 padding='same',
                                 activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(inputs=conv2,
                                        pool_size=[2,2],
                                        strides=2)
        
        conv3 = tf.layers.conv2d(inputs=pool2,
                                 filters=64,
                                 kernel_size=[5,5],
                                 padding='same',
                                 activation=tf.nn.relu)
        pool3 = tf.layers.max_pooling2d(inputs=conv3,
                                        pool_size=[2,2],
                                        strides=2)
        
        pool3_flatten = tf.layers.flatten(pool3)
        dense = tf.layers.dense(inputs=pool3_flatten,
                                units=128,
                                activation=tf.nn.relu)
        dropout = tf.layers.dropout(inputs=dense,
                                    rate=0.1,
                                    training=is_training)
        logits = tf.layers.dense(inputs=dropout,
                                units=10)
    return is_training, logits

In [9]:
is_training, logits = cnn_model(batch_imgs, batch_labels)
with tf.name_scope('metrics'):
    cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=batch_labels, logits=logits))
    correct_prediction = tf.equal(tf.argmax(logits, 1), batch_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)

init = tf.global_variables_initializer()

train_steps = 500
with tf.Session() as sess:
    sess.run(init)
    sess.run(iterator.initializer)
    for i in range(train_steps):
        cross_entropy_val, acc_val, _ = sess.run([cross_entropy, accuracy, train_op],
                                                 feed_dict = {
                                                     is_training: True
                                                 })

        if (i+1) % 50 == 0:
            print '[Train] Step %4d: ce: %4.5f, acc: %4.5f' % (i+1, cross_entropy_val, acc_val)

[Train] Step   50: ce: 1.97382, acc: 0.20000
[Train] Step  100: ce: 0.90876, acc: 0.65000
[Train] Step  150: ce: 0.23699, acc: 1.00000
[Train] Step  200: ce: 0.05548, acc: 1.00000
[Train] Step  250: ce: 0.01611, acc: 1.00000
[Train] Step  300: ce: 0.02590, acc: 1.00000
[Train] Step  350: ce: 0.00244, acc: 1.00000
[Train] Step  400: ce: 0.00237, acc: 1.00000
[Train] Step  450: ce: 0.18420, acc: 0.95000
[Train] Step  500: ce: 0.05949, acc: 1.00000
